<center><img src="https://i.imgur.com/zRrFdsf.png" width="700"></center> 

_____

<a id='home'></a>

# Merging

<a target="_blank" href="https://colab.research.google.com/github/CienciaDeDatosEspacial/code_and_data/blob/main/Merging_DFs.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

Merging data sets need the following considerations:

* Merging is done on two data frames.
* You need a column in each data frame that share the same exact and unique values. The column names or titles need not be the same.
* The merged table shows by default the mutual coincidences; but you can also request the values not matched, which will help you detect possible extra cleaning.
* Pandas jargon uses a **left** and a **right** data frame: **left**.merge(**right**).

At this stage, let me use other data frames we prepared previously:

In [ ]:
import pandas as pd

co2Link='https://github.com/CienciaDeDatosEspacial/dataSets/raw/main/co2.csv'
forestLink='https://github.com/CienciaDeDatosEspacial/dataSets/raw/main/forestRev.csv'

co2=pd.read_csv(co2Link)
forest=pd.read_csv(forestLink)

Remember the amount of rows of each DF:

In [ ]:
co2.shape,forest.shape

Also keep in mind the column names:

In [ ]:
forest.columns,co2.columns

Let me show you some merge approaches, but I will only show the amount of columns produced:

1. You keep only what is common in both key columns:

This is the default. The final rows will be the ones where the key values in each data frame match exactly. In this case, your count of rows will be at most the amount of rows of the smallest data frame.

In [ ]:
# how many resulting rows after inner merging
co2.merge(forest,how='inner',left_on='name',right_on='Country').shape

2. You keep all the keys from one data frame:

The final rows will be all the rows from the dataframe (here from the _left_). If a key values does not find a match, the key value is kept, but the columns will have missing values. In this case, your count of rows will be equal to the amount of rows of the data frame to the left. You can also use **right** so the same logic applies to the data frame to the right.



In [ ]:
# how many resulting rows after left merging
co2.merge(forest,how='left',left_on='name',right_on='Country').shape

3. You keep all the rows from both data frames:

In this case you will obtain all possible rows: the matched values, and the unmatched values from both data frames. You will also generate missing values. In this case, your count of rows will be at least the amount of rows of the data frame with the most rows.


In [ ]:
# how many resulting rows after outer merging
co2.merge(forest,how='outer',left_on='name',right_on='Country').shape

Why the different amount of rows? 

In [ ]:
set(co2.name)-set(forest.Country)

In [ ]:
set(forest.Country)-set(co2.name)

Apparently, the data is not available from every country. So, let's just continue:

In [ ]:
# the default is inner merge
cia=co2.merge(forest,left_on='name',right_on='Country')
cia

Let's bring back the data on fragility, but just for the year 2019:

In [ ]:
import os

#read in:
FragilityAll=pd.read_csv(os.path.join("data","fragility.csv"))

#subset
fragile2019=FragilityAll.loc[FragilityAll.Year==2019,:"Total"].copy()

# see
fragile2019

We will practice **fuzzy merging** now.

In [ ]:
# Countries in 'cia' but NOT in 'fragile2019' 
OnlyCia=set(cia.Country)-set(fragile2019.Country)
OnlyCia

In [ ]:
# Countris in 'fragile2019' but NOT in 'cia' 
OnlyFragile=set(fragile2019.Country)-set(cia.Country)
OnlyFragile

Here, we should try to find the what countries in _OnlyFragile_ may match the ones in _OnlyCia_. We need to use the **fuzzy merge** approach (please install **thefuzz** if not previously installed):

In [ ]:
from thefuzz import process as fz

# take a country from OnlyFragile
# look for a country in OnlyCia and return the most similar
[(f,fz.extractOne(f, OnlyCia)) for f in sorted(OnlyFragile)]

Above you have found _some_ good matches. Let's keep the best ones:

In [ ]:
[(f,fz.extractOne(f, OnlyCia)) for f in sorted(OnlyFragile)
 if fz.extractOne(f, OnlyCia)[1]>=87]

Once you have good matches, you have to create dictionary like this:

In [ ]:
changesFragile1={f:fz.extractOne(f, OnlyCia)[0] 
                 for f in sorted(OnlyFragile)
                 if fz.extractOne(f, OnlyCia)[1] >=87}
#dict of matches
changesFragile1

You can use that dict for the replacements:

In [ ]:
fragile2019.Country.replace(to_replace=changesFragile1,inplace=True)

Now the countries in fragile2019 have more matches. 

This process can be done a few more times, and you can recover more rows for the merging process. Let's see:

In [ ]:
# second try
OnlyCia=set(cia.Country)-set(fragile2019.Country)
OnlyFragile=set(fragile2019.Country)-set(cia.Country)
[(f,fz.extractOne(f, OnlyCia)) for f in sorted(OnlyFragile)]

In [ ]:
# second dict of changes
# select a different threshold
changesFragile2={f:fz.extractOne(f, OnlyCia)[0] 
                 for f in sorted(OnlyFragile)
                 if 74<=fz.extractOne(f, OnlyCia)[1]}

#dict of matches
changesFragile2

In [ ]:
# make the changes
fragile2019.Country.replace(to_replace=changesFragile2,inplace=True)

In [ ]:
# third try
OnlyCia=set(cia.Country)-set(fragile2019.Country)
OnlyFragile=set(fragile2019.Country)-set(cia.Country)
[(f,fz.extractOne(f, OnlyCia)) for f in sorted(OnlyFragile)]

In [ ]:
# third dict of changes
# new threshold
changesFragile3={f:fz.extractOne(f, OnlyCia)[0] 
                 for f in sorted(OnlyFragile)
                 if 54==fz.extractOne(f, OnlyCia)[1]}

#dict of matches
changesFragile3

In [ ]:
# make changes
fragile2019.Country.replace(to_replace=changesFragile3,inplace=True)

In [ ]:
# fourth try

OnlyCia=set(cia.Country)-set(fragile2019.Country)
OnlyFragile=set(fragile2019.Country)-set(cia.Country)
[(f,fz.extractOne(f, OnlyCia)) for f in sorted(OnlyFragile)]

The fourth attempt did not offer good results. So we are ready:

In [ ]:
fragilecia=fragile2019.merge(cia) #merge on Country
fragilecia

In [ ]:
#checking:
fragilecia.info()

Merging is a key process for producing analytics. So, it is always good to add some 'standard' information to avoid the need of fuzzy merging. See this data table


In [ ]:
isoLink='https://github.com/CienciaDeDatosEspacial/dataSets/raw/main/isodata.csv'
isoCodes=pd.read_csv(isoLink)
isoCodes.head()

We should add the **ISO** columns to our recent merged data frame:

In [ ]:
# key columns are not spelled the same:
isoCodes.Countryname=isoCodes.Countryname.str.upper()
isoCodes.merge(fragilecia,left_on='Countryname',right_on='Country')

We have lost several countries, then we redo the fuzzy merge:

In [ ]:
onlyFrcia=set(fragilecia.Country)-set(isoCodes.Countryname)
onlyISO=set(isoCodes.Countryname)-set(fragilecia.Country)

[(f,fz.extractOne(f, onlyISO)) for f in sorted(onlyFrcia)]

Prepare changes:

In [ ]:
# first change
changesFrcia1={f:fz.extractOne(f, onlyISO)[0] 
                 for f in sorted(onlyFrcia)
                 if fz.extractOne(f, onlyISO)[1] >=87}
#dict of matches
changesFrcia1

In [ ]:
# make changes
fragilecia.Country.replace(to_replace=changesFrcia1,inplace=True)

Second iteration

In [ ]:
onlyFrcia=set(fragilecia.Country)-set(isoCodes.Countryname)
onlyISO=set(isoCodes.Countryname)-set(fragilecia.Country)

[(f,fz.extractOne(f, onlyISO)) for f in sorted(onlyFrcia)]

This second iteration gives weird results. Let's use a different function to get more than one result:

In [ ]:
onlyFrcia=set(fragilecia.Country)-set(isoCodes.Countryname)
onlyISO=set(isoCodes.Countryname)-set(fragilecia.Country)

[(f,fz.extract(f, onlyISO)) for f in sorted(onlyFrcia)]

In [ ]:
# remember you can use this for a particular case:
isoCodes.loc[isoCodes.Countryname.str.contains('LAO')]

Then, just prepare manual changes:

In [ ]:
lastChanges={'CONGO, REPUBLIC OF THE':'CONGO (THE)',
 'KOREA, SOUTH':'KOREA (THE REPUBLIC OF)',
'LAOS':"LAO PEOPLE'S DEMOCRATIC REPUBLIC (THE)"}

fragilecia.Country.replace(to_replace=lastChanges,inplace=True)

Then,

In [ ]:
fragciaiso=isoCodes.merge(fragilecia,left_on='Countryname',right_on='Country')
fragciaiso.head()

In [ ]:
fragciaiso.drop(columns=['Country','name'],inplace=True)
fragciaiso.rename(columns={'Countryname':"Country",'Year':'fragility_date','Total':'fragility'},inplace=True)
fragciaiso

Let's save what we have:

In [ ]:
fragciaiso.to_csv(os.path.join("data","FragilityCia_isos.csv"), index=False)